# GRPO Training on SQuAD Adversarial Prompter

This notebook follows Hugging Face's GRPO guide for TRL and adapts it to our dataset and reward functions. See reference: [Post training an LLM for reasoning with GRPO in TRL](https://huggingface.co/learn/cookbook/en/fine_tuning_llm_grpo_trl).



In [1]:
import logging
import os

# Export path
import sys

from datasets import Dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer

sys.path.append("/Users/sonphat.tran/grpo_adv_prompter_squad")

from src.utils.datasets import load_csv_dataset
from src.utils.rewards import bert_reward, format_reward

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("grpo_training")

# Paths and config
DATASET_CSV = "/Users/sonphat.tran/grpo_adv_prompter_squad/data/squad_golden.csv"
OUTPUT_DIR = "/Users/sonphat.tran/grpo_adv_prompter_squad/outputs/qwen2-0.5b-grpo"
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"

os.makedirs(OUTPUT_DIR, exist_ok=True)
logger.info("Output dir: %s", OUTPUT_DIR)

/Users/sonphat.tran/grpo_adv_prompter_squad/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:grpo_training:Output dir: /Users/sonphat.tran/grpo_adv_prompter_squad/outputs/qwen2-0.5b-grpo


In [6]:
# 1) Load dataset and adapt to conversation format expected by TRL GRPO
raw_dataset: Dataset = load_csv_dataset(DATASET_CSV)
raw_dataset = raw_dataset.shuffle(seed=42)
train_dataset = raw_dataset.select(range(min(2000, len(raw_dataset))))
logger.info("Train rows: %d", len(train_dataset))

# Sanity check columns used by rewards
required_cols = [
    "id",
    "context",
    "question",
    "answer",
    "answer_start_char_idx",
    "answer_end_char_idx",
    "prompt",
]
missing = [c for c in required_cols if c not in train_dataset.column_names]
if missing:
    raise ValueError(f"Missing columns for rewards: {missing}")

# 2) Load base model and tokenizer
# We apply LoRA for memory efficiency per HF guide
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype="auto",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Setup LoRA
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
logger.info("Model with LoRA ready")

INFO:grpo_training:Train rows: 2000
INFO:grpo_training:Model with LoRA ready


In [ ]:
# 3) Define GRPO config (adapted from HF guide)
training_args = GRPOConfig(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    remove_unused_columns=False,  # keep extra columns for reward kwargs
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    bf16=True,
    per_device_train_batch_size=1,
    # Preprocessing controls
    max_completion_length=256,  # shorter for quick runs
    num_generations=4,
    max_prompt_length=512,
    # Logging and saving
    report_to=["tensorboard"],
    logging_steps=1,
    save_strategy="steps",
    save_steps=50,
)
training_args

GRPOConfig(output_dir='/Users/sonphat.tran/grpo_adv_prompter_squad/outputs/qwen2-0.5b-grpo', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=1e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='/Users/sonphat.tran/grpo_adv_prompter_squad/outputs/qwen2-0.5b-grpo/runs/Sep19_16-52-24_ITVN004103-MAC', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, loggin

In [4]:
# 4) Instantiate GRPOTrainer with both rewards
# format_reward expects the assistant completion content to match <think>...</think><answer>...</answer>
# bert_reward uses the dataset columns to compute QA-based penalties

reward_funcs = [format_reward, bert_reward]

trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_funcs,
    args=training_args,
    train_dataset=train_dataset,
)
trainer

In [ ]:
# 5) Train and save
train_result = trainer.train()
logger.info("Training complete: %s", str(train_result))

trainer.save_model(OUTPUT_DIR)
logger.info("Saved to %s", OUTPUT_DIR)